install the necessary Python packages

In [2]:
!pip install transformers
!pip install pytube moviepy openai-whisper ffmpeg-python
!pip install yt-dlp


In [3]:
!pip install librosa

1. Download YouTube Video
The script uses yt-dlp to download the best quality video and audio from a given YouTube URL. The user is prompted to input the YouTube URL, and the video is downloaded and saved as video.mp4.

2. Extract and Resample Audio
Using ffmpeg, the audio is extracted from the downloaded video, saved as ytaudio.wav, and then resampled to 16000 Hz, saved as ytaudio_16k.wav.


In [5]:
import yt_dlp
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC, pipeline
import torch
import soundfile as sf
import librosa
from IPython.display import Audio

# Function to download YouTube video
def download_video(youtube_url, output_file='video.mp4'):
    try:
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': output_file,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(youtube_url, download=True)
            print(f"Video downloaded successfully: {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")

# User input for YouTube URL
youtube_url = input("Enter YouTube URL: ")
download_video(youtube_url)

# Extract audio from the downloaded video
!ffmpeg -i video.mp4.webm -q:a 0 -map a ytaudio.wav



Enter YouTube URL: https://youtu.be/rwF-X5STYks?si=QH6VY2MeUYJ6F1vi
[youtube] Extracting URL: https://youtu.be/rwF-X5STYks?si=QH6VY2MeUYJ6F1vi
[youtube] rwF-X5STYks: Downloading webpage
[youtube] rwF-X5STYks: Downloading ios player API JSON
[youtube] rwF-X5STYks: Downloading tv player API JSON
[youtube] rwF-X5STYks: Downloading m3u8 information
[info] rwF-X5STYks: Downloading 1 format(s): 303+251
[download] Destination: video.mp4.f303.webm
[download] 100% of   23.58MiB in 00:00:00 at 36.95MiB/s  
[download] Destination: video.mp4.f251.webm
[download] 100% of    2.02MiB in 00:00:00 at 25.72MiB/s  
[Merger] Merging formats into "video.mp4.webm"
Deleting original file video.mp4.f303.webm (pass -k to keep)
Deleting original file video.mp4.f251.webm (pass -k to keep)
Video downloaded successfully: video.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.


3. Transcribe Audio
The Wav2Vec2 model from Facebook is loaded to transcribe the audio. The transcription is printed to the console.

4. Summarize Transcription
The transcription is summarized using the summarization pipeline from transformers library. The summary is printed to the console.

In [6]:

# Load pre-trained model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Resample audio to 16000 Hz
!ffmpeg -i ytaudio.wav -ar 16000 ytaudio_16k.wav

# Load and play the resampled audio
speech, rate = librosa.load("ytaudio_16k.wav", sr=16000)
# audio = Audio("ytaudio_16k.wav", autoplay=True)

# # Display the audio player
# audio

# Transcribe audio
input_values = tokenizer(speech, return_tensors='pt').input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.decode(predicted_ids[0])

# Print transcription
print("Transcription:")
print(transcription)

# Load summarization pipeline
summarizer = pipeline("summarization")

# Summarize the transcription
summary = summarizer(transcription, max_length=200, min_length=50, do_sample=False)

# Print summary
print("\nSummary:")
print(summary[0]['summary_text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]


Summary:
 NEW CONTENT IS CREATED by ARTIFICIAL INTELLIGENCE ITS called GENERATIVE A I THIS COULD INVOLVE GENERATING TEXTS AND D IMAGES as well as VIDIOS MUSIC OR VOICES TO DO this you DESCRIBE in a CHAT DIALOGUE BOX WHAT YOU WANT THE A I TO CREATE THIS DESCRIPTION IS CALLED ARE PROMPT The A I ToOLS PROVide ANSWERS TO ALL Sorts of QUESTIONS SUMMARIZED COMPLEX INFORMATION AND GENERATE DIVERSE IDEAS .
